In [89]:
# Imports

import pandas as pd


In [90]:
# Funções auxiliares

def verificarSeSaoTodosNumeros(dataFramePNumerico):
    todos_numeros = pd.to_numeric(dataFramePNumerico['ano'], errors='coerce').notna().all()
    print(f'Todos os valores na coluna "ano" são números? {todos_numeros}')

In [91]:
# Upload dos datasets

marcas_de_carro = [
    'Acura', 'AlfaRomeo', 'AMGeneral', 'Aston Martin', 'Audi', 'Bentley', 'BMW', 'GMC', 'Toyota', 'Volkswagen',
    'honda', 'Bugatti', 'Buick', 'Cadillac', 'Chevrolet', 'Chrysler', 'Daewoo', 'Dodge', 'Eagle', 'Ferrari',
    'FIAT', 'Fisker', 'Ford', 'Genesis', 'Geo', 'HUMMER', 'Hyundai', 'INFINITI', 'Isuzu', 'Jaguar', 'Jeep',
    'Kia', 'Lamborghini', 'Land Rover', 'Lexus', 'Lincoln', 'Lotus', 'Maserati', 'Maybach', 'Volvo', 'Tesla',
    'Suzuki', 'Subaru', 'Spyker', 'smart', 'Scion', 'Saturn', 'Mazda', 'McLaren', 'Mercedes-Benz', 'Mercury',
    'MINI', 'Mitsubishi', 'Nissan', 'Oldsmobile', 'Panoz', 'Plymouth', 'Pontiac', 'Porsche', 'Ram', 'Rolls-Royce',
    'Saab'
]

for marca in marcas_de_carro:
    file_path = f'./Data/Scrapped_Car_Reviews_{marca}.csv'
    # df = pd.read_csv(file_path)
    # df = pd.read_csv(file_path, delimiter='\t')  # Substitua '\t' pelo delimitador correto
    df = pd.read_csv(file_path, encoding='utf-8')


    # Dropar colunas desnecessarias
    colunas_para_dropar = ['Author_Name', 'Review_Date', 'Review_Title', 'Review']
    df = df.drop(columns=colunas_para_dropar)

    # Criar ano de acordo com a primeira string
    df['ano'] = df['Vehicle_Title'].str.extract('(\d{4})')
    df['marca'] = df['Vehicle_Title'].str.split().str[1]
    df['categoria'] = df['Vehicle_Title'].str.extract(r'(\b\w+\b)\s*\(')


    verificarSeSaoTodosNumeros(df)
    df.head()



ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.
